In [2]:
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import col
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [8]:
#creditcard_data = sqlContext.read.csv('..//creditcardfraud/creditcard',header='true', inferSchema='true')
data = sqlContext.read.format("csv") .option("header", "true") .option("inferSchema", "true") .load("creditcardfraud/creditcard.csv")

In [9]:
print(data.take(4))#.foreach(println)

[Row(Time=Decimal('0'), V1=-1.3598071336738, V2=-0.0727811733098497, V3=2.53634673796914, V4=1.37815522427443, V5=-0.338320769942518, V6=0.462387777762292, V7=0.239598554061257, V8=0.0986979012610507, V9=0.363786969611213, V10=0.0907941719789316, V11=-0.551599533260813, V12=-0.617800855762348, V13=-0.991389847235408, V14=-0.311169353699879, V15=1.46817697209427, V16=-0.470400525259478, V17=0.207971241929242, V18=0.0257905801985591, V19=0.403992960255733, V20=0.251412098239705, V21=-0.018306777944153, V22=0.277837575558899, V23=-0.110473910188767, V24=0.0669280749146731, V25=0.128539358273528, V26=-0.189114843888824, V27=0.133558376740387, V28=-0.0210530534538215, Amount=149.62, Class=0), Row(Time=Decimal('0'), V1=1.19185711131486, V2=0.26615071205963, V3=0.16648011335321, V4=0.448154078460911, V5=0.0600176492822243, V6=-0.0823608088155687, V7=-0.0788029833323113, V8=0.0851016549148104, V9=-0.255425128109186, V10=-0.166974414004614, V11=1.61272666105479, V12=1.06523531137287, V13=0.4890

In [103]:
train_data_orig , test_data_orig = data.randomSplit([0.8, 0.2])

In [10]:
data.printSchema()

root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double

In [137]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,   LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from time import time

In [106]:
def transformedData(input_data):
    return input_data.rdd.map(lambda row: LabeledPoint(row[-1], row[:-1]))

In [107]:
#transformed_data = transformedData(data)

In [108]:
#display(transformed_data)#.printSchema()

In [109]:
train_data = transformedData(train_data_orig)
test_data = transformedData(test_data_orig)

In [147]:
modelA = LogisticRegressionWithLBFGS.train(train_data)

In [148]:
pred  = test_data.map(lambda p: (p.label, modelA.predict(p.features)))   

In [149]:
display(pred)

PythonRDD[782] at RDD at PythonRDD.scala:48

In [150]:
display(pred.groupBy("prediction")) 

PythonRDD[787] at RDD at PythonRDD.scala:48

In [38]:
pred.take(4)

[(0.0, 0), (0.0, 1), (0.0, 0), (0.0, 0)]

In [65]:
pred.top(5)

[(1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1)]

In [151]:
test_accuracy = pred.filter(lambda vp: vp[0] == vp[1]).count() / float(test2.count())

In [152]:
tp = pred.filter(lambda vp: vp[0] == 1.0 and vp[1] ==1).count()
fp = pred.filter(lambda vp: vp[0] == 0.0 and vp[1] ==1).count()
tn = pred.filter(lambda vp: vp[0] == 0.0 and vp[1] ==0).count()
fn = pred.filter(lambda vp: vp[0] == 1.0 and vp[1] ==0).count()

In [153]:
print(test_accuracy)
print(tp)
print(fp)
print(tn)
print(fn)
print(tp+fp+tn+fn)

0.998771002686237
63
33
56824
37
56957


In [154]:
print("Precision = ", float(tp/(tp+fp)))
print("Recall = ", float(tp/(tp+fn)))
print("Accuracy = ", float((tp+tn)/(tp+fp+tn+fn)))

Precision =  0.65625
Recall =  0.63
Accuracy =  0.998771002686237


In [155]:
test = test_data.toDF()
test2= test.select("label")

In [156]:
test2.printSchema()
test2.take(5)
test2.count()
pred.count()

root
 |-- label: double (nullable = true)



56957

In [157]:
tt=test.filter((col("label") == 1.0))

In [158]:
tt.count()

100

In [159]:
train_data_reduced = train_data_orig .drop('Time').drop('V28').drop('V27').drop('V26').drop('V25').drop('V24').drop('V23') .drop('V22').drop('V20').drop('V15').drop('V13').drop('V8')
test_data_reduced = test_data_orig .drop('Time').drop('V28').drop('V27').drop('V26').drop('V25').drop('V24').drop('V23') .drop('V22').drop('V20').drop('V15').drop('V13').drop('V8')

In [160]:
train_data_red = transformedData(train_data_reduced)
test_data_red = transformedData(test_data_reduced)

In [161]:
modelB = LogisticRegressionWithLBFGS.train(train_data_red)
pred_red  = test_data_red.map(lambda p: (p.label, modelB.predict(p.features)))   

In [162]:
test_accuracy_red = pred_red.filter(lambda vp: vp[0] == vp[1]).count() / float(test_data_red.count())
test_accuracy_red

0.9977175764172973

In [163]:
tp_r = pred_red.filter(lambda vp: vp[0] == 1.0 and vp[1] ==1).count()
fp_r = pred_red.filter(lambda vp: vp[0] == 0.0 and vp[1] ==1).count()
tn_r = pred_red.filter(lambda vp: vp[0] == 0.0 and vp[1] ==0).count()
fn_r = pred_red.filter(lambda vp: vp[0] == 1.0 and vp[1] ==0).count()

In [164]:
print(test_accuracy_red)
print(tp_r)
print(fp_r)
print(tn_r)
print(fn_r)
print(tp_r+fp_r+tn_r+fn_r)

0.9977175764172973
58
88
56769
42
56957


In [165]:
print("Precision = ", float(tp_r/(tp_r+fp_r)))
print("Recall = ", float(tp_r/(tp_r+fn_r)))
print("Accuracy = ", float((tp_r+tn_r)/(tp_r+fp_r+tn_r+fn_r)))

Precision =  0.3972602739726027
Recall =  0.58
Accuracy =  0.9977175764172973
